In [1]:
import numpy as np
import pandas as pd
import datetime
import math

In [6]:
dam = pd.read_csv('dam.csv')
dam

,market,date,hour,price,volume
0,1,01/01/2016,0,40.10,477.5
1,1,01/01/2016,1,40.08,486.6
2,1,01/01/2016,2,40.06,495.6
3,1,01/01/2016,3,40.08,496.6
4,1,01/01/2016,4,40.07,502.9
...,...,...,...,...,...
44587,1,29/01/2021,13,61.97,134.0
44588,1,29/01/2021,23,58.05,59.0
44589,1,30/01/2021,17,42.08,64.0
44590,1,31/01/2021,3,42.19,149.0


In [12]:
dam['date']=pd.to_datetime(dam['date'],format="%d/%m/%Y")#"%d/%m/%Y")
dam=dam.set_index(pd.DatetimeIndex(dam['date']))
dam['year']=dam.index.year
dam['month']=dam.index.month
dam['day']=dam.index.day
dam['day_of_week']=dam.index.dayofweek
dam['month_no']=dam['year']+dam['month']/100
dam['hour_ref'] = dam['day_of_week'].astype(str) + '_' + dam['hour'].astype(str)
dam

,market,date,hour,price,volume,year,month,day,day_of_week,month_no,hour_ref
date,,,,,,,,,,,
2016-01-01,1,2016-01-01,0,40.10,477.5,2016,1,1,4,2016.01,4_0
2016-01-01,1,2016-01-01,1,40.08,486.6,2016,1,1,4,2016.01,4_1
2016-01-01,1,2016-01-01,2,40.06,495.6,2016,1,1,4,2016.01,4_2
2016-01-01,1,2016-01-01,3,40.08,496.6,2016,1,1,4,2016.01,4_3
2016-01-01,1,2016-01-01,4,40.07,502.9,2016,1,1,4,2016.01,4_4
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-29,1,2021-01-29,13,61.97,134.0,2021,1,29,4,2021.01,4_13
2021-01-29,1,2021-01-29,23,58.05,59.0,2021,1,29,4,2021.01,4_23
2021-01-30,1,2021-01-30,17,42.08,64.0,2021,1,30,5,2021.01,5_17


In [13]:
monthly=dam.groupby(['month_no','year','month','hour', 'hour_ref']).mean().reset_index(['hour','month','year'])
monthly=monthly.drop(['day'],1)
monthly.rename(columns={'price': 'price_mean'}, inplace=True)
monthly['volume_sum']=dam[['month_no','hour_ref','volume']].groupby(['month_no','hour_ref']).sum()
monthly['hours']=dam[['month_no','hour_ref']].groupby(['month_no','hour_ref']).size()
monthly['power']=monthly['volume_sum']/monthly['hours']
monthly

year  month  hour  market  price_mean   volume  \
month_no hour_ref                                                   
2016.01  0_0       2016      1     0     1.0     49.8775  390.500   
         1_0       2016      1     0     1.0     48.2125  311.100   
         2_0       2016      1     0     1.0     47.9725  327.875   
         3_0       2016      1     0     1.0     49.3950  306.225   
         4_0       2016      1     0     1.0     47.3560  327.400   
...                 ...    ...   ...     ...         ...      ...   
2021.01  2_23      2021      1    23     1.0     47.2225  139.025   
         3_23      2021      1    23     1.0     51.0150  122.000   
         4_23      2021      1    23     1.0     53.4500  109.220   
         5_23      2021      1    23     1.0     48.0280  121.800   
         6_23      2021      1    23     1.0     47.0520  117.880   

                   day_of_week  volume_sum  hours    power  
month_no hour_ref                                           
2016.01  0_0               0.0      1562.0      4  390.500  
         1_0               1.0      1244.4      4  311.100  
         2_0               2.0      1311.5      4  327.875  
         3_0               3.0      1224.9      4  306.225  
         4_0               4.0      1637.0      5  327.400  
...                        ...         ...    ...      ...  
2021.01  2_23              2.0       556.1      4  139.025  
         3_23              3.0       488.0      4  122.000  
         4_23              4.0       546.1      5  109.220  
         5_23              5.0       609.0      5  121.800  
         6_23              6.0       589.4      5  117.880  

[10248 rows x 10 columns]

month_no  hour_ref
2016.01   0_0         4
          0_1         4
          0_10        4
          0_11        4
          0_12        4
                     ..
2021.01   6_5         5
          6_6         5
          6_7         5
          6_8         5
          6_9         5
Length: 10248, dtype: int64

In [15]:
monthly.to_csv('dam_monthly_hours.csv')

In [52]:
dam.to_csv('dam_out.csv')